# Introduction to Machine Learning
## Lesson 4 Lasso, Ridge and Cross Validation

## Introduction


The task of our first lesson is to understand how work Lasso, Ridgee and Cross Validation. This includes going through the theory and following a set of instructions to complete the code.


## Task

In this lab work we will learn and try to understand cross validation, lasso and ridge. A simplified version of the cab dataset will be used as the dataset

### About Taxi Dataset
Taxi trip records include id, vendor_id, passenger_count, store_and_fwd_flag, distance_km, log_trip_duration.
The dataset is often used for testing algorithms for classification and pattern recognition due to its simplicity and well-defined structure.

### Performing the Classification
To do so you will need:
- Obtain data from competition
- Create a Jupyter notebook which will produce a file for submission
- Submit it to the competition

### Questions with answers

*Choose the correct statements about Lasso and Ridge*: 

1. Ridge-regularization is more likely to zero out model weights than Lasso.
2. Lasso regularization is more likely to zero out model weights than Ridge.
3. Ridge and Lasso regularization are designed to deal with the situation of undertraining. 
4. The essence of Lasso regularization is to add to the minimized functional the sum of the modules of the trained coefficients.
5. The essence of Lasso regularization is to add to the minimized functional the sum of squares of the trained coefficients.



**Answer: 2, 4)** Both types of regularization, as it was shown in the lecture, can be reduced to the search of a conditional extremum on some compact set. In the case of Ridge regularization, this set turns out to be convex (for example, in two-dimensional space it is a circle), exactly like the minimized functional. Then the chance that the level line and the boundary of the set from the constraint will touch somewhere on the axis is rather small. In the case of Lasso, the situation is the opposite (see Visual explanation in the lecture). 
_______________________________________________________

*Choose the correct statements about Multicollinearity: 

1. Multicollinearity ensures that we get an overfitted model.
2. Multicollinearity occurs when there is a strong linear dependence in the object-attribute matrix.
3. If there are linearly dependent features in the object-attribute matrix, then we cannot apply the matrix formula for finding the optimal regression coefficients with a 50% probability.
4. If linearly dependent features are present in the object-sign matrix, then the minimized functional has not one single but 2-3 points of minimum.
5. If the object-sign matrix contains linearly dependent features, then the minimized functional has an infinite number of minima.
6. Multicollinearity can be cured by removing dependent features or using regularization.




**Answer: 2, 5, 6)** it is important to realize that this reasoning applies to the problem of linear regression just like in the lecture back. For other models, multicollinearity may, generally speaking, not lead to any problems.
_______________________________________________________

## Importing required Libraries

First we need to import necessary libraries:

[Pandas](https://pandas.pydata.org/) - For data analysis and manipulation

[Numpy](https://numpy.org/) - To deal with matrices

Then drop unused fields.

In [2]:
import numpy as np
import pandas as pd

## Preparaing Data
Preparing data for machine learning involves several steps such as data collection, cleaning the data from noise and outliers, transforming the data into a suitable format, normalizing or standardizing values, and creating and selecting features (feature engineering) to improve the quality of the model. This process is important to ensure the accuracy and reliability of machine learning models because data quality directly affects their performance.

In [3]:
processed_data = pd.read_csv('processed_data.csv', index_col='id')
processed_data = processed_data.assign(log_trip_duration=np.log1p(processed_data['trip_duration']))
processed_data = processed_data.drop('trip_duration', axis=1)
processed_data.head()

,vendor_id,passenger_count,store_and_fwd_flag,distance_km,log_trip_duration
id,,,,,
id2875421,1,930.399753,0,1.500479,6.122493
id2377394,0,930.399753,0,1.807119,6.498282
id3858529,1,930.399753,0,6.392080,7.661527
id3504673,1,930.399753,0,1.487155,6.063785
id2181028,1,930.399753,0,1.189925,6.077642


#### Exercise 1
The task is to measure the quality of Linear Regression after processing the data on Cross-Validation on 4 folds

use LinearRegression and cross_validate from sklearn

In [49]:
from sklearn.model_selection import KFold

selector = KFold(n_splits=4, shuffle=True, random_state=33)

from sklearn.model_selection import cross_validate
from sklearn.linear_model import LinearRegression

model = LinearRegression()

X = processed_data.drop('log_trip_duration', axis=1)
Y = processed_data['log_trip_duration']

cv_result_1 = cross_validate(model, X, Y, 
                             scoring='neg_mean_squared_error',
                             cv=selector, return_train_score=True)

cross_val_error = -np.mean(cv_result_1['test_score'])

print(f"RMSLE with Cross Validation: {round(cross_val_error, 3)}")

RMSLE with Cross Validation: 0.425


### Task 2

#### Exercise 2

In linear algebra the concept of **rank of a matrix** is often used. It corresponds to the number of linearly independent columns in the matrix. In other words, it allows us to estimate whether there is an excess of information in our dataframe. If the rank of the matrix is less than the number of used columns, some fixtures should be removed, because otherwise a situation of strict multicollinearity occurs.

To measure the rank in our matrixes, we can use the function numpy.linalg.matrix_rank

The constant feature can be neglected in this exercise.

You should output it in console through function of python **print**

In [50]:
print(f"Total features: {X.shape[1]}, - rank equal {np.linalg.matrix_rank(X)}")

Total features: 4, - rank equal 4


#### Exercise 3
Doesn't it seem to us that because of the new 4 features there is a problem of multicollinearity? How to make it so that, on the one hand, we get adequate quality, and on the other hand not to remove new features?

Regularization will help us here

Find such a regularization parameter $\lambda$ for the Ridge and Lasso case that the RMSLE error on cross-validation is strictly less than 0.4

**ALARM**: use a data mass-scaling procedure (use the MinMaxScaler method) before applying the regularization. Important - to preserve the concept of independence of training on the traine and on the test, at each iteration of cross-validation it is necessary to measure the standardization parameters exclusively on the traine and then apply them on the validation fold.

In [51]:
from sklearn.preprocessing import MinMaxScaler

from sklearn.pipeline import Pipeline

from sklearn.linear_model import Lasso, Ridge

from sklearn.model_selection import GridSearchCV

In [52]:
### Add scaling as a step in Pipeline and train lasso

pipe_lasso = Pipeline([('scaler', MinMaxScaler()),
                       ('lasso', Lasso(max_iter=100000))])

alphas = np.linspace(0.00001, 0.0001, num=30)

#### Let's go through a bunch of degrees of regularization
### Notice that we'll find the best ones that have better quality,
### Than the required quality

cv_lasso = GridSearchCV(pipe_lasso, param_grid={'lasso__alpha':alphas},
                        scoring='neg_mean_squared_error', cv=selector,
                        return_train_score=True)

cv_lasso.fit(X, Y)

GridSearchCV(cv=KFold(n_splits=4, random_state=33, shuffle=True),
             estimator=Pipeline(steps=[('scaler', MinMaxScaler()),
                                       ('lasso', Lasso(max_iter=100000))]),
             param_grid={'lasso__alpha': array([1.00000000e-05, 1.31034483e-05, 1.62068966e-05, 1.93103448e-05,
       2.24137931e-05, 2.55172414e-05, 2.86206897e-05, 3.17241379e-05,
       3.48275862e-05, 3.79310345e-05, 4.10344828e-05, 4.41379310e-05,
       4.72413793e-05, 5.03448276e-05, 5.34482759e-05, 5.65517241e-05,
       5.96551724e-05, 6.27586207e-05, 6.58620690e-05, 6.89655172e-05,
       7.20689655e-05, 7.51724138e-05, 7.82758621e-05, 8.13793103e-05,
       8.44827586e-05, 8.75862069e-05, 9.06896552e-05, 9.37931034e-05,
       9.68965517e-05, 1.00000000e-04])},
             return_train_score=True, scoring='neg_mean_squared_error')

Find the best model on Cross-Validation. Use **best_estimator_** from **cv_lasso**

In [43]:
cv_lasso.best_estimator_

Pipeline(steps=[('scaler', MinMaxScaler()),
                ('lasso', Lasso(alpha=1e-05, max_iter=100000))])

List of different scorers for all models
The ones listed above. Use **cv_results_**

In [53]:
### Список различных скоров для всех моделей
### Которые были перебраны выше

cv_lasso.cv_results_['mean_test_score']

array([-0.42529285, -0.42530117, -0.4253109 , -0.42532203, -0.42533456,
       -0.42534849, -0.42536384, -0.42538058, -0.42539873, -0.42541828,
       -0.42543924, -0.4254616 , -0.42548537, -0.42551054, -0.42553711,
       -0.42556509, -0.42559447, -0.42562526, -0.42565745, -0.42569105,
       -0.42572605, -0.42576245, -0.42580026, -0.42583947, -0.42588008,
       -0.4259221 , -0.42596553, -0.42601035, -0.42605659, -0.42610422])

#### Exercise 4
In this assignment, adjust the hyperparameters of the Ridge regression model using cross-validation. Create a payplane with MinMaxScaler and Ridge, define a grid of alpha values, use GridSearchCV with negative RMS error and cross validation. Then apply fit to X and Y data to find the optimal hyperparameters.

In [45]:
### Ridge

pipe_ridge = Pipeline([('scaler', MinMaxScaler()), ('ridge', Ridge())])

cv_ridge = GridSearchCV(pipe_ridge, param_grid={'ridge__alpha':alphas},
                        scoring='neg_mean_squared_error', cv=selector,
                        return_train_score=True)

cv_ridge.fit(X, Y)

GridSearchCV(cv=KFold(n_splits=4, random_state=33, shuffle=True),
             estimator=Pipeline(steps=[('scaler', MinMaxScaler()),
                                       ('ridge', Ridge())]),
             param_grid={'ridge__alpha': array([1.00000000e-05, 1.31034483e-05, 1.62068966e-05, 1.93103448e-05,
       2.24137931e-05, 2.55172414e-05, 2.86206897e-05, 3.17241379e-05,
       3.48275862e-05, 3.79310345e-05, 4.10344828e-05, 4.41379310e-05,
       4.72413793e-05, 5.03448276e-05, 5.34482759e-05, 5.65517241e-05,
       5.96551724e-05, 6.27586207e-05, 6.58620690e-05, 6.89655172e-05,
       7.20689655e-05, 7.51724138e-05, 7.82758621e-05, 8.13793103e-05,
       8.44827586e-05, 8.75862069e-05, 9.06896552e-05, 9.37931034e-05,
       9.68965517e-05, 1.00000000e-04])},
             return_train_score=True, scoring='neg_mean_squared_error')

 Find the optimal hyperparameters of the Ridge regression model using cross-validation. Use **best_estimator_** from **cv_ridge**

In [46]:
cv_ridge.best_estimator_

Pipeline(steps=[('scaler', MinMaxScaler()), ('ridge', Ridge(alpha=1e-05))])

Output the mean values of the test data scores for each combination of parameters. Use **cv_results_** like in previous exercise 3

In [47]:
cv_lasso.cv_results_['mean_test_score']

array([-0.42529285, -0.42530117, -0.4253109 , -0.42532203, -0.42533456,
       -0.42534849, -0.42536384, -0.42538058, -0.42539873, -0.42541828,
       -0.42543924, -0.4254616 , -0.42548537, -0.42551054, -0.42553711,
       -0.42556509, -0.42559447, -0.42562526, -0.42565745, -0.42569105,
       -0.42572605, -0.42576245, -0.42580026, -0.42583947, -0.42588008,
       -0.4259221 , -0.42596553, -0.42601035, -0.42605659, -0.42610422])